In [50]:
import os
from dotenv import load_dotenv

MODEL= "llama2"

In [51]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model= Ollama(model=MODEL)
embeddings= OllamaEmbeddings()

In [52]:
from langchain_core.output_parsers import StrOutputParser

parser= StrOutputParser()

chain=model | parser
chain.invoke("Make me laugh")

"\nOkay, here's a silly joke for you:\n\nWhy don't lobsters share?\n\nBecause they're shellfish! 😜\n\nI hope that made you smile at least! Do you have any specific topics or subjects you'd like me to make jokes about?"

Langchain will talk to the model, send a request to the model, we then get an o/p from the model and we then pipe that o/p into the i/p of parser--> which will then return the string.

In [53]:
#Loading the document and splitting it:

from langchain_community.document_loaders import PyPDFLoader

loader= PyPDFLoader("Example.pdf")     #I have used the research paper which introduced RAG as an example.a
pages= loader.load_and_split()     #for efficient processing and better integration.
pages

[Document(metadata={'source': 'Example.pdf', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research; ‡University College London; ⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate knowl-\nedge is still limited, and hence on knowledge-intensive tasks, their performance\nlags behind task-speciﬁc architectures. Additionally, providing provenance for their\ndecisions and updating their world knowledge remain open research problems. Pre-\ntrained models with a differentiable access mechanism to explicit

In [54]:
#Adding a template or a prompt:

from langchain.prompts import PromptTemplate

template= """Answer the question based on the context below. If you can't answer the question,
reply with "I don't know". 

Context: {context}

Question: {question} """

prompt= PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context.", question="Here is a qustion."))


Answer the question based on the context below. If you can't answer the question,
reply with "I don't know". 

Context: Here is some context.

Question: Here is a qustion. 


In [55]:
#Now we need to pass out 'prompt' to the model.
#How do we do that? --> by building up the chain.

chain= prompt |model|parser

In [56]:
chain.input_schema.schema()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12880\3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [58]:
#This is just an example to see how the chain works:
chain.invoke(
    {
        "context": "I was born on 2002, it's 2024 now.",
        "question": "What is my age?"
    }
)

"Of course! Based on the context you provided, your current age is 22 years old (born in 2002 and it's 2024)."

In [33]:
#Now, we need to find a way to take out 'document' and pass it as a 'context':
#We don't need the whole document, just the relevant parts.

from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore= DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

c:\Users\ASUS\OneDrive\Desktop\Folder\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
retriever=vectorstore.as_retriever()     #the context needed for answering the prompt is coming from the retriever.
vectorstore.as_retriever().invoke("RAG")

[Document(metadata={'source': 'Example.pdf', 'page': 10}, page_content='[7] Christopher Clark and Matt Gardner. Simple and Effective Multi-Paragraph Reading Compre-\nhension. arXiv:1710.10723 [cs], October 2017. URL http://arxiv.org/abs/1710.10723.\narXiv: 1710.10723.\n[8] Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. BERT: Pre-training of\nDeep Bidirectional Transformers for Language Understanding. In Proceedings of the 2019 Con-\nference of the North American Chapter of the Association for Computational Linguistics: Human\nLanguage Technologies, Volume 1 (Long and Short Papers), pages 4171–4186, Minneapolis,\nMinnesota, June 2019. Association for Computational Linguistics. doi: 10.18653/v1/N19-1423.\nURL https://www.aclweb.org/anthology/N19-1423.\n[9] Emily Dinan, Stephen Roller, Kurt Shuster, Angela Fan, Michael Auli, and Jason Weston. Wiz-\nard of wikipedia: Knowledge-powered conversational agents. In International Conference on\nLearning Representations, 2019. 

In [59]:
from operator import itemgetter

chain=(
    {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")
    }
    |prompt
    |model
    |parser 
) 

chain.invoke({"question": "What is NLP?"})


"\nNatural Language Processing (NLP) is a subfield of artificial intelligence that deals with the interaction between computers and human language. It involves the development of algorithms and statistical models that enable computers to process, understand, and generate natural language data, such as text, speech, and gestures.\n\nThe goals of NLP include:\n\n1. Language understanding: Developing algorithms that can understand and interpret natural language texts, including their meaning, context, and intent.\n2. Language generation: Creating models that can generate natural language texts, such as text summaries, translations, or chatbots.\n3. Sentiment analysis: Determining the emotional tone or sentiment of a piece of text, whether it's positive, negative, or neutral.\n4. Named entity recognition: Identifying and categorizing named entities in text, such as people, organizations, and locations.\n5. Part-of-speech tagging: Identifying the part of speech (such as noun, verb, adjectiv

In [44]:
questions=[
    "What is this document about?",
    "When was RAG introduced?",
    "Is RAG better than existing models?"
]
 
for question in questions:
    print(f"Question: {question}")
    print(f"Answer:{chain.invoke({"question": question})}")
    print()

Question: What is this document about?
Answer:This document appears to be a research paper discussing the use of retrieval-augmented generation (RAG) models for knowledge-intensive NLP tasks. The authors argue that pre-trained language models have limitations when it comes to accessing and manipulating knowledge, and propose a hybrid approach that combines parametric and non-parametric memories. They introduce RAG models, which combine pre-trained seq2seq models with dense vector indexes of Wikipedia, accessed with a pre-trained neural retriever. The authors evaluate their models on a range of knowledge-intensive NLP tasks and set the state of the art on three open domain QA tasks, outperforming parametric-only seq2seq baselines. They also discuss the potential benefits of using RAG models, including more specific, diverse, and factual language generation.

Question: When was RAG introduced?
Answer:RAG was introduced in 2020 by the authors of the paper "Stack-Augmented Recurrent Neural